In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 
import sys
sys.path.append("/home/cat/code/widefieldPredict/") # Adds higher directory to python modules path.


import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
from tqdm import tqdm
import pandas as pd
import scipy
from tqdm import tqdm, trange

import os
import time
import warnings
warnings.filterwarnings("ignore")

from scipy import signal
from scipy.fft import fftshift
from scipy.signal import savgol_filter
from numpy.polynomial import polynomial as P
from scipy.signal import argrelmax


from locanmf import LocaNMF, postprocess
from locanmf import analysis_fig4 
from visualize import visualize_svm




Autosaving every 180 seconds


ModuleNotFoundError: No module named 'locanmf'

In [2]:

def load_raw_data(spatial_fname, temporal_fname):
    # GRAB AND RECONSTRUCT DATA
    spatial = np.load(spatial_fname) 
    temporal = np.load(temporal_fname)
    temporal = temporal.transpose(0,2,1)

    #
    print (spatial.shape)
    print (temporal.shape)

    #
    print ("reconstructing data: ")
    data = np.matmul( temporal, spatial)
    print (data.shape)

    #
    print ("getting mean of data: ")
    data_mean = data.mean(0)
    print ("data_mean: ", data_mean.shape)
    # compute variance in raw data- not used
    # var = np.var(data2d, axis=0)
    # print ("var: ", var.shape)

    ######################################
    ###### COMPUTE RAW ROI ACTIVITY ######
    ######################################
    data2D = data_mean.reshape(data_mean.shape[0], 128,128)
    print ("Data mean 2D: ", data2D.shape)

    # 
    means = []
    ctr=0
    for id_ in ordered_names:
        area_id = areas[id_]
        idx = np.where(atlas==area_id)
        print (ctr, "areaId: ", area_id, names[id_], idx[0].shape)
        mask = np.zeros((128,128),'float32') + np.nan
        mask[idx] = 1

        temp = data2D*mask
        roi = np.nanmean(temp, axis=1)
        roi = np.nanmean(roi, axis=1)
        means.append(roi)

        ctr+=1   

    #
    raw_means = np.array(means)
    print ("Raw data means: ", raw_means.shape)

    return raw_means



def load_locaNMF_data(fname_locaNMF):
    # order locaNMF components by plot color ORDER in Fig 4A
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                          13,2,
                          12,3,
                          11,4,
                          10,5,
                          9,6,
                          8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        #print ("file missing", fname_locaNMF)
        return None, None, None, None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    #print ("locanmf data: ", locaNMF_temporal.shape)

    # 
    areas = d['areas'][ordered_names]
    names = d['names'][ordered_names]
    #print ("original names: ", names.shape)

    #
    atlas = np.load('/home/cat/code/widefieldPredict/locanmf/atlas_fixed_pixel.npy')
    #print ("atlas: ",atlas.shape)
    # print (areas)
    # print (names)

    #print ("  # of ordered_names: ", ordered_names.shape)
    #print ("ORDERED NAMES: ", names[ordered_names])

    
    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random
  
        
# #
def compute_variance_locaNMF(locaNMF_temporal):
    # 
    t = np.arange(locaNMF_temporal.shape[2])/30 - 30
    means = []
    var = []
    #for k in ordered_names:
    for k in range(locaNMF_temporal.shape[1]):
        temp = locaNMF_temporal[:,k].mean(0)
        means.append(temp)

        # 
        temp = np.var(locaNMF_temporal[:,k],axis=0)
        var.append(temp)

    #
    means = np.array(means)[:,:900]
    var = np.array(var)[:,:900]
    #print (means.shape, var.shape)

    return means, var


def plot_variance_locaNMF(var):
    colors = plt.cm.jet(np.linspace(0,1,var.shape[0]))

    scale1 = 1
    scale2 = .005
    linewidth = 2
    # scale3 = np.nan
    t = np.arange(var.shape[1])/30-30
    #print ("t: ", t)
    fig=plt.figure(figsize=(10,10))
    for k in range(var.shape[0]):
        ax=plt.subplot(121)
        plt.xlim(-15,0)
        plt.xticks([])
        plt.yticks([])

        # 
        temp = var[k]
        temp = temp*scale1+k*scale2

        # 
        #plt.plot(t,temp,'--', c=colors[k],
        plt.plot(t,temp, c=colors[k],
                linewidth=linewidth)

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c=colors[k],alpha=.5)

    if False:
        plt.savefig('/home/cat/variance.png',dpi=300)
        plt.close()
    else:
        plt.show()
  

def load_variances(animal_id):
    session_name = 'all'
    root_dir = '/media/cat/4TBSSD/yuki/'

    # 
    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("# of sessions: ", loca.sessions.shape)

    # 
    loca.all_means = []
    loca.saved_names = []
    loca.all_means_random = []
    loca.good_sessions = []
    loca.all_vars = []
    loca.all_vars_random = []
    loca.all_n_trials = []
    
    # all trial curves
    loca.temporal = []
    loca.random = []
    
    # 
    for session in tqdm(loca.sessions):

        # load data
        fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                     session + '_locanmf.npz')
        atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

        # 
        if atlas is not None:
            means, var = compute_variance_locaNMF(locaNMF_temporal)
            loca.all_n_trials.append(locaNMF_temporal.shape[0])
            loca.all_means.append(means)
            loca.all_vars.append(var)
            # 
            means_r, var_r = compute_variance_locaNMF(random)
            loca.all_means_random.append(means_r)
            loca.all_vars_random.append(var_r) 

            loca.saved_names = names
            loca.good_sessions.append(session)
            
            #
            loca.temporal.append(locaNMF_temporal)
            loca.random.append(random)

    # 
    
    return loca

############################################################
def compute_and_plot_variance_peaks(var, 
                                   means,
                                   n_trials,
                                   saved_names,
                                   session_id,
                                   plotting=False,
                                   smoothing=False):
    
    # 
    colors = plt.cm.jet(np.linspace(0,1,var.shape[0]))

    scale1 = 1.5E-4
    scale2 = 1E-6
    
    linewidth = 2

    # 
    if plotting:
        fig = plt.figure(figsize=(10,10))
        ax1 = plt.subplot(3,1,1)
        ax2 = plt.subplot(3,1,2)
        plt.xticks(np.arange(16), saved_names, 
                   rotation='vertical',
                   fontsize=14)
        #
        ax3 = plt.subplot(3,1,3)
        
        ax1.set_xlim(-30,0)
        
    # 
    ratios = []
    times = []
    # loop over all ROIs
    for k in range(var.shape[0]):

        # 
        temp = var[k]
        
        if True:
            temp3 = temp                    
        else:
            temp2 = means[k]

            temp2 += 100

            temp3 = temp/temp2
            temp3 = temp3*scale1+k*scale2

            # 
        if smoothing:
            window_length = 9
            polyorder = 3
            temp3 = scipy.signal.savgol_filter(temp3, 
                                               window_length, polyorder)
                                       
        # 
        if plotting:
            t = np.arange(temp3.shape[0])/30-30

            ax1.plot(t,temp3*scale1+k*scale2, c=colors[k],
                     linewidth=linewidth)

            ax1.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.5)
        
        # compute mean and plot it
        trace_control = np.mean(temp3[:temp3.shape[0]//2])
        if plotting:
            ax1.plot([-30,0], [trace_control*scale1+k*scale2,
                           trace_control*scale1+k*scale2],'--',c=colors[k], alpha=.5)

        
        #
        min_val = np.min(temp3[temp3.shape[0]//2:])
        ratio = min_val/trace_control
        
        # NOTE WE ONLY CONSIDER last 5 sec of data and exlcuding last 500ms
        #arg_min = np.argmin(temp3[temp3.shape[0]//2:-15])
        # print ('temp3 shape: ', temp3.shape)
        #arg_min = np.argmin(temp3[750:900-15])
        
        
        
        #arg_min = np.argmin(temp3[750:900-15])
        arg_min = np.argmin(temp3[450:900])
        arg_min = arg_min/30.-15

        
        
        # 
        if plotting:
            arg_min_int = int(arg_min*30+15)
            
            ax1.scatter(t[arg_min_int+temp3.shape[0]//2],
                        temp3[arg_min_int+temp3.shape[0]//2]*scale1+k*scale2, 
                        #color=colors[k],
                        color='black',
                        s=100)
        
        ratios.append(ratio)
        times.append(arg_min)

        if plotting:
            ax2.scatter(k, ratio,
                        s=300,
                       #color=colors[k]
                        color='black'
                       )
            
            ax2.set_ylim(0,1.0)
            ax2.set_xlim(-0.5,15.5)

            if k==0:
                for p in range(0,100,25):
                    ax2.plot([-0.5,16], [p/100.,p/100.],'--', c='black',alpha=.5)
                
            ax3.scatter(k, arg_min,
                        s=300,
                        #color=colors[k]
                        color='black',
                        )
            
            ax3.set_ylim(-5.0,0)
            ax3.set_xlim(-0.5,15.5)

            if k==0:
                for p in range(-5,1,1):
                    ax3.plot([-0.5,16],[p,p],'--', c='black',alpha=.5)
            #ax3.set_xticks([])
            #ax3.set_yticks([])
        
        # 
        if False:
            print (k, " area ", saved_names[k], 
               " ratio change: ", round(ratio,2),
               " time: ", round(arg_min,2))
        
        #return 
# 
    if plotting:
        plt.suptitle(animal_id+"  session: "+str(session_id) + 
             "    n trials: "+str(n_trials))

        if True:
            fig.savefig('/home/cat/variance.svg',dpi=300)
            #fig2.savefig('/home/cat/ratios.svg',dpi=300)
            #fig3.savefig('/home/cat/times.svg',dpi=300)
            plt.close('all')
        else:
            plt.show()

    return ratios, times

# 
#    
def plot_histograms_ratios(all_ratios, all_times, color,
                          min_ratio_threshold = 0.15):
    min_ratio_threshold = 0.15
    bin_width = .25
    bins = np.arange(-5,0.5,bin_width)
    good_vals = []

    plotting = False
    for k in range(all_ratios.shape[0]):
        idx = np.where(all_ratios[k]<min_ratio_threshold)[0]

        good_times = all_times[k][idx]
        good_vals.append(good_times)

        if plotting:
            y = np.histogram(good_times,bins = bins)

            plt.plot(y[1][1:],y[0],
                    c = colors[k])

    good_vals = np.hstack(good_vals)
    y = np.histogram(good_vals,bins = bins)

    plt.plot(y[1][1:],y[0],
            c = color)
    

    
def plot_variance_peaks_3_plots(all_vars, all_means, all_n_trials,
                                saved_names, plotting,
                                session_id = None,
                                smoothing = False):
    
    
    if session_id is None:
        session_ids = np.arange(len(all_vars))
        if plotting==True:
            print (" CAN't plot all sesssions, exiting")
            return
    else:
        session_ids = [session_id]

    
    all_ratios = []
    all_times = []
    for session_id in session_ids:
        # session_id = 3
        # print (session_id, "n_trials: ", all_n_trials[session_id])

        # 
        ratios, times = compute_and_plot_variance_peaks(all_vars[session_id], 
                                               all_means[session_id],
                                               all_n_trials[session_id],
                                               saved_names,
                                               session_id,
                                               plotting,
                                               smoothing)
        all_ratios.append(ratios)
        all_times.append(times)

        # if plotting input sessoin_id manually and use break
        # break

    all_ratios = np.array(all_ratios)
    all_times = np.array(all_times)
    print (all_times.shape)
    print ("DONE")
    
    return all_ratios, all_times


def plot_box_plots_times(all_ratios, all_times, 
                         min_ratio_threshold=0.15):
    
    
    codes = ['Retrosplenial', 'barrel', 'limb', 'visual','motor']
    #codes = ['limb, layer 1 - right', 'limb, layer 1 - left']
    clrs_local = ['black','blue','red','magenta', 'pink','brown']
    
    # 
    bin_width = .25
    bins = np.arange(-5,0.5,bin_width)

    # 
    edts = []
    for a in range(len(all_ratios)):
        good_vals = []
        for k in range(all_ratios[a].shape[0]):
            idx = np.where(all_ratios[a][k]<min_ratio_threshold)[0]

            if idx.shape[0]>0:
                good_times = all_times[a][k][idx]
                good_vals.append(good_times)

            # 
        good_vals = np.hstack(good_vals)

        edts.append(good_vals)
        
    print ("M2: ", edts[1])
    
    # 
    my_dict = dict(M1 = edts[0], 
                   M2 = edts[1],
                   M3 = edts[2],
                   M4 = edts[3],
                   M5 = edts[4],                      
                   M6 = edts[5]
                     )

    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    # 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)


    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
#         plt.plot(x, y, 
#                  mfc = clrs_local[i], 
#                  mec='k', 
#                  ms=7, 
#                  marker="o", 
#                  linestyle="None",
#                     )
        #
        colors = plt.cm.viridis(np.linspace(0,1,len(edts[i])))
        x = np.random.normal(i+1, 0.04, len(edts[i]))
        print (i,d, ' y shape: ', y.shape)
        plt.scatter(x, edts[i], 
                   #c=clrs_local[i],
                   c=colors,
                   edgecolor='black',
                   s=200,
                   #alpha=np.linspace(.2, 1.0, x.shape[0])
                   alpha=.5
                   )

        
        
        res = scipy.stats.normaltest(edts[i])
        print ("i: ", i, '  res: ', res)
        
#     pvals = [0.05,0.01,0.001,0.0001,0.00001]
#     patches = []
#     if len(codes)!=2:
#         for p in [0,2,4]:
#             res = stats.ks_2samp(edts_saved[p], edts_saved[3])
#             label_ = ''
#             for k in range(len(pvals)):
#                 if res[1]<pvals[k]:
#                     label_ = label_ + "*"
#                 else:
#                     break

#                 patches.append(mpatches.Patch(color='blue', label=label_))



def load_locanmf_traces(fname_locaNMF):
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                          13,2,
                          12,3,
                          11,4,
                          10,5,
                          9,6,
                          8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        #print ("file missing", fname_locaNMF)
        return None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    
    return locaNMF_temporal, locaNMF_temporal_random
    
# 

def transform_data(data1, data2, method, window):

    # 
    if method=='correlation1':
        data2 = pd.Series(data2)
        data1 = pd.Series(data1)
        res1 = data1.rolling(window).corr(data2)
        
    elif method=='correlation2':
       # print (s1.shape, data.shape)
        res1 = scipy.signal.signaltools.correlate(data1, data2, mode='same')
    
    elif method=='cov':
        
        #res1 = np.cov(s1, data, rowvar=True)
        res1 = np.corrcoef(data1, data2, rowvar=False)
        print (res1)
    
    return res1


def compute_correlation_curves(loca, window, session_id, animal_id):
  

    overwrite= False
    # 
    area_ids = np.arange(16)
    methods = ['correlation1', 'correlation2']
    
    for method in methods: 
            
        prefix3 = '_'+method+'_'

        fname_out = os.path.join('/media/cat/4TBSSD/yuki',animal_id,'tif_files', loca.sessions[session_id],
                             loca.sessions[session_id]+'_code_04'+prefix3+
                             'trial_ROItimeCourses_30sec_pca_0.95.npy')

        if os.path.exists(fname_out) and overwrite==False:
            continue
            
        # 
        fname_locaNMF = os.path.join('/media/cat/4TBSSD/yuki',animal_id,'tif_files', 
                                     loca.sessions[session_id],
                                     loca.sessions[session_id]+'_locanmf.npz')
        
        trials, random = load_locanmf_traces(fname_locaNMF)
        #print ("temporal: ", temporal.shape, random.shape)
        if trials is None:
            continue
        
        # 
        trial_all = np.zeros((trials.shape), dtype=np.float32)
        random_all = np.zeros((random.shape), dtype=np.float32)
        for area_id in tqdm(area_ids, desc=method):
            #if area_id != 9:
            #    continue
            sums = []
            sums_random = []
            #s1 = loca.temporal[session_id][:,area_id]
            s1 = trials[:,area_id] 
            res1_array = []
            res2_array = []
            for k in range(s1.shape[0]):

                # compute mean but leave out the current trial
                temp0 = np.delete(s1, k, axis=0).copy()
                mean = pd.Series(np.median(temp0,axis=0))
                std = np.std(temp0,axis=0)
               
                # 
                temp1 = trials[k,area_id]
                res1 = transform_data(temp1,mean, method)
                trial_all[k,area_id]= res1

                ##################
                temp2 = random[k,area_id]
                res2 = transform_data(temp2,mean, method)
                random_all[k,area_id]= res2
                
                if area_id == 9:
                    ax=plt.subplot(1,2,1)
                    plt.plot(temp1, c='grey',alpha=.1)
                    #plt.plot(temp2, c='red',alpha=.1)
                    
                    ax=plt.subplot(1,2,2)
                    plt.plot(res1, c='grey',alpha=.1)
                    plt.plot(res2+2, c='red',alpha=.1)
            
            if area_id==9:
                ax=plt.subplot(1,2,1)
                plt.plot(s1.mean(0), c='black')
                plt.plot(trial_all[:,area_id].mean(0),'--', c='black')
                plt.plot(random_all[:,area_id].mean(0),'--', c='red')
                
                ax=plt.subplot(1,2,2)
                plt.plot(s1.mean(0), c='black')
            
        ############################################################
        # replace the nans with random chunks
        for k in range(trials.shape[0]):
            for p in range(trials.shape[1]):
                trial_all[k,p,:30] = trial_all[k,p,30:60]

        for k in range(trials.shape[0]):
            for p in range(trials.shape[1]):
                random_all[k,p,:30] = random_all[k,p,30:60]


        np.save(fname_out, trial_all)
        np.save(fname_out.replace('trial','random'), random_all)


In [3]:
# ###########################################################
# ########### LOAD RAW DATA OPTIONAL - FIG 4A ONLY ##########
# ###########################################################

# spatial_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy'
# temporal_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
    
# raw_means = load_raw_data(spatial_fname, temporal_fname)

# Plot locaNMF vs. raw
# plot_locanmf_vs_raw(locaNMF_temporal, raw_means)

In [4]:
# ######################################################
# ###### LOAD LOCANMF DATA AND CORRECT ORDER MAPS ######
# ######################################################

# # 
# animal_id = 'IJ1'
# loca = load_variances(animal_id)


In [41]:
##################################################
##################################################
##################################################
window = 30

fig=plt.figure()
# 
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']    
animal_ids = ['IJ1']
for animal_id in animal_ids:

    # load trials and random time courses
    loca = load_variances(animal_id)
    
    # 
    session_ids = np.arange(len(loca.temporal))
    for session_id in session_ids:
        compute_correlation_curves(loca, window, session_id, animal_id)

plt.show()


  7%|▋         | 3/44 [00:00<00:01, 23.53it/s]

# of sessions:  (44,)


100%|██████████| 44/44 [00:01<00:00, 22.58it/s]

['Primary motor area, Layer 1 - right'
 'Primary motor area, Layer 1 - left'
 'Secondary motor area, layer 1 - right'
 'Secondary motor area, layer 1 - left'
 'Primary somatosensory area, barrel field, layer 1 - right'
 'Primary somatosensory area, barrel field, layer 1 - left'
 'Primary somatosensory area, lower limb, layer 1 - right'
 'Primary somatosensory area, lower limb, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - right'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary visual area, layer 1 - right'
 'Primary visual area, layer 1 - left'
 'Retrosplenial area, lateral agranular part, layer 1 - right'
 'Retrosplenial area, lateral agranular part, layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - right'
 'Retrosplenial area, dorsal part, layer 1 - left']


In [18]:
##################################################
##################################################
##################################################

fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_correlation2_trial_ROItimeCourses_30sec_pca_0.95.npy'
data = np.load(fname)
print (data.shape)
plt.plot(data[:,0,:900].T,
        c='black',
        alpha=.2)

fname2 = fname.replace('trial','random')
print ("fname2: ", fname2)
data = np.load(fname2)
print (data.shape)
plt.plot(data[:,0,:900].T+2,
        c='red',
        alpha=.2)

plt.show()


(61, 16, 1801)
fname2:  /media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_correlation2_random_ROItimeCourses_30sec_pca_0.95.npy
(61, 16, 1801)


In [67]:
print (trials.shape)
#print (t)
fig=plt.figure()
plt.plot(trials[0].T)
plt.show()

(114, 16, 1801)


In [55]:
d = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print (d.shape)

(61, 7, 1801)


In [ ]:
# ########################################
# s1 = pd.Series(res1)

# ########################################
# analytic_signal = hilbert(ss1)
# amplitude_envelope = np.abs(analytic_signal)

# instantaneous_phase = np.unwrap(np.angle(analytic_signal))
# instantaneous_frequency = (np.diff(instantaneous_phase) /
#                            (2.0*np.pi) * fs)

# # 
# ax=plt.subplot(2,2,2)
# plt.plot(t, amplitude_envelope)
# plt.plot(t, ss1)
# plt.title("amplitude envelope")

# ax=plt.subplot(2,2,3)
# plt.plot(t,instantaneous_phase)
# plt.title("instateanous phase")

# ax=plt.subplot(2,2,4)
# plt.plot(t[1:],instantaneous_frequency)
# plt.title("instateanous freq")

In [165]:
###############################
###### VARIANCE ANALYSIS ######
###############################

# FIG 6 H variance curves + times + ratios

plotting = True  # note you need to run plotting=True for a single sessoin not all!!!
session_id = -1

saved_names = [
'M1-R',
'M1-L',
'M2-R',
'M2-L',
'SomB-R',
'SomB-L',
'SomH-R',
'SomH-L',
'SomF-R',
'SomF-L',
'V1-R',
'V1-L',
'RL-R',
'RL-L',
'RD-R',
'RD-L'    
]

smoothing = False
all_ratios, all_times = plot_variance_peaks_3_plots(all_vars, all_means, all_n_trials,
                                                    saved_names, plotting,
                                                    session_id,
                                                    smoothing)

(1, 16)
DONE


In [166]:
#############################################
###### ALL ANIMAL LOWEST VARIANCE TIME ######
#############################################

animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']    
#animal_ids = ['IA3']
session_id = None

# 
ratios = []
times= []
smoothing = True
for ctr,animal_id in enumerate(animal_ids):
    all_means, all_vars, saved_names, all_n_trials= load_variances(animal_id)

    plotting = False  # note you need to run plotting=True for a single sessoin not all!!!
    all_ratios, all_times = plot_variance_peaks_3_plots(all_vars, all_means, all_n_trials,
                                                        saved_names, 
                                                        plotting,
                                                        session_id,
                                                        smoothing)
    ratios.append(all_ratios)
    times.append(all_times)
    

fig = plt.figure(figsize=(10,10))
min_ratio_threshold = 0.25
plot_box_plots_times(ratios, times, 
                     min_ratio_threshold)
plt.ylim(-15,0)

if True:
    plt.savefig('/home/cat/min_var_times.svg')
    plt.close()
else:
    plt.show()



  4%|▍         | 3/71 [00:00<00:02, 29.41it/s]

# of sessions:  (71,)


  9%|▉         | 4/44 [00:00<00:01, 25.72it/s]

(53, 16)
DONE
# of sessions:  (44,)


  5%|▍         | 2/44 [00:00<00:02, 17.95it/s]

...

(40, 16)
DONE
# of sessions:  (44,)


 14%|█▎        | 15/110 [00:00<00:00, 131.63it/s]

(39, 16)
DONE
# of sessions:  (110,)


100%|██████████| 110/110 [00:07<00:00, 14.60it/s]


(77, 16)
DONE
M2:  [ -0.03333333  -0.03333333  -0.03333333  -0.23333333  -0.2
  -0.2         -0.16666667  -0.03333333  -5.4        -11.03333333
 -11.03333333 -13.8        -13.8        -13.8        -13.8
 -13.8        -13.8        -11.9        -11.9         -4.9
  -4.86666667  -4.96666667  -4.96666667  -0.06666667  -0.1
  -7.03333333  -0.06666667  -0.03333333  -0.03333333  -1.5
  -6.2         -0.1         -0.03333333  -0.03333333  -0.03333333
  -6.76666667  -9.76666667 -12.73333333  -9.73333333  -1.3
  -7.76666667  -1.3         -1.23333333  -1.3         -3.1
  -3.03333333  -2.06666667  -1.86666667  -1.8         -3.8
  -2.1         -0.03333333  -5.66666667  -8.06666667  -7.83333333
  -7.83333333]
0 M1  y shape:  (310,)
i:  0   res:  NormaltestResult(statistic=71.55154354564556, pvalue=2.9025501570615157e-16)
1 M2  y shape:  (310,)
i:  1   res:  NormaltestResult(statistic=10.522011119232783, pvalue=0.005190083155821618)
2 M3  y shape:  (310,)
i:  2   res:  NormaltestResult(statistic=300.0

In [37]:

data = np.load('/media/cat/4TBSSD/yuki/IA3/tif_files/IA3am_Mar15_30Hz/IA3am_Mar15_30Hz_locanmf.npz',
              allow_pickle=True)

A_reshape = data['A_reshape']
temp = data['temporal_trial']
names = data['areanames_area']

print (temp.shape)
mean = temp.mean(0)


fig=plt.figure()
for i in range(A_reshape.shape[2]):
    plt.subplot(4,4,i+1)
    plt.imshow(A_reshape[:,:,i])
    plt.xticks([])
    plt.yticks([])
    plt.title(names[i],fontsize=6)
plt.tight_layout(h_pad=0.5,w_pad=0.5)
plt.show()




(34, 16, 1801)


In [62]:
############################################


def get_sessions(main_dir,
                 animal_id,
                 session_id):
     # load ordered sessions from file
    sessions = np.load(os.path.join(main_dir,
                                         animal_id,
                                         'tif_files.npy'))
    # grab session names from saved .npy files
    data = []
    for k in range(len(sessions)):
        data.append(os.path.split(sessions[k])[1].replace('.tif',''))
    sessions = data

    #
    if session_id != 'all':
        final_session = []
        for k in range(len(sessions)):
            if session_id in sessions[k]:
                final_session = [sessions[k]]
                break
        sessions = final_session

    # fix binary string files issues; remove 'b and ' from file names
    for k in range(len(sessions)):
        sessions[k] = str(sessions[k]).replace("'b",'').replace("'","")
        if sessions[k][0]=='b':
            sessions[k] = sessions[k][1:]

    sessions = np.array(sessions)

    return sessions

main_dir = '/media/cat/4TBSSD/yuki/'
session_id = 'all'


    
animal_ids = ['AQ2']
fig =plt.figure(figsize=(10,10))
ax=plt.subplot()
xmax = 21
for animal_id in animal_ids: 
    sessions = get_sessions(main_dir,
                             animal_id,
                             session_id)
    # 
    colors = plt.cm.plasma(np.linspace(0,1,len(sessions)))
    for ctr,session in enumerate(sessions):
        #print ("session: ", session)

        # 
        fname_var = os.path.join(main_dir,animal_id,'tif_files',session,
                                             session+ '_whole_stack_trial_ROItimeCourses_15sec_pca_var_explained.npy')

        if os.path.exists(fname_var)==True:
        
            var = np.load(fname_var)

            var = np.cumsum(var)
            x=np.arange(var.shape[0])+1
            plt.plot(x[:xmax], 
                     var[:xmax],
                    color=colors[ctr],
                    linewidth=2,
                    alpha=.9)
            
            
            
        
#
plt.plot([0,xmax],[0.95,0.95],
        '--',c='black')
#plt.semilogy()
plt.ylim(.6,1)
plt.xlim(0,xmax-1)

cmap = plt.cm.plasma  # define the colormap

cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
cb = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, spacing='proportional')
cb.set_label('gdp_md_est')




if True:
    plt.savefig('/home/cat/variance.svg', dpi=1200)
    plt.close()
else:
    plt.show()
        
    
    
    

In [28]:
d = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print (d.shape)

(106, 6, 1801)


In [38]:
data = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components_spatial.npy')
print (data.shape)

mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ1/genericmask.txt'))

mask1 = np.ones((128,128),'float32')

for k in range(mask.shape[0]):
    mask1[mask[k][0],
          mask[k][1]] = np.nan
    
print (mask1.shape)

(30, 16384)
(128, 128)


In [41]:
for k in range(6):
    ax=plt.subplot(2,3,k+1)
    temp = data[k].reshape(128,128)*mask1
    plt.imshow(temp)
    
plt.savefig('/home/cat/pcas.svg',dpi=1200)
plt.close()